In [2]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape

(2674080, 6)

In [3]:
np.save("data/btc_1m.npy", trading_1m)

In [1]:
import numpy as np
from jesse import helpers, research

trading_1m = np.load("data/btc_1m.npy")
trading_1m.shape

(2674080, 6)

In [2]:
from custom_indicators.toolbox.dollar_bar import estimate_dollar_bar_threshold

for i in [10, 45, 60 * 4]:
    threshold = estimate_dollar_bar_threshold(trading_1m, i)
    print(f"{i}m: {threshold}")

10m: 71968406.59988706
45m: 379150625.6237953
240m: 2078307128.56747


In [3]:
from custom_indicators.config import DOLLAR_BAR_THRESHOLD_10M, DOLLAR_BAR_THRESHOLD_4H
from custom_indicators.toolbox.dollar_bar import build_dollar_bar

dollar_bar_10m = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_10M)
dollar_bar_4h = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_4H)
print(dollar_bar_10m.shape, dollar_bar_4h.shape)

(268413, 6) (11226, 6)


In [4]:
np.save("data/dollar_bar_10m.npy", dollar_bar_10m)
np.save("data/dollar_bar_4h.npy", dollar_bar_4h)

In [6]:
from custom_indicators.config import DOLLAR_BAR_THRESHOLD_45M
from custom_indicators.toolbox.dollar_bar import build_dollar_bar

candles_dollar_bar = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_45M)
candles_dollar_bar.shape

(59138, 6)

In [7]:
from custom_indicators.toolbox.labeler.zigzag_labeler import zigzag

res = zigzag(candles_dollar_bar[:, 2], 0.01)
np.unique(res, return_counts=True)

(array([-1,  1]), array([28757, 30381]))

In [8]:
label = np.column_stack((candles_dollar_bar, res))
label

array([[ 1.57784172e+12,  7.18872000e+03,  7.18559000e+03, ...,
         7.17400000e+03,  5.26204210e+04, -1.00000000e+00],
       [ 1.57792146e+12,  7.18559000e+03,  7.10740000e+03, ...,
         7.10100000e+03,  5.31224620e+04, -1.00000000e+00],
       [ 1.57797306e+12,  7.10767000e+03,  6.96029000e+03, ...,
         6.92200000e+03,  5.41026110e+04, -1.00000000e+00],
       ...,
       [ 1.73826966e+12,  1.05086300e+05,  1.04836200e+05, ...,
         1.04768000e+05,  3.81996600e+03, -1.00000000e+00],
       [ 1.73827080e+12,  1.04836300e+05,  1.05080900e+05, ...,
         1.04737000e+05,  3.65874900e+03, -1.00000000e+00],
       [ 1.73827656e+12,  1.05080800e+05,  1.04676900e+05, ...,
         1.04628300e+05,  3.65185700e+03, -1.00000000e+00]])

In [9]:
np.save("data/label_dollar_bar_45m.npy", label)

In [6]:
from jesse import utils

df_dollar_bar = utils.numpy_candles_to_dataframe(candles_dollar_bar, name_date="time")
df_dollar_bar["zigzag"] = res
df_dollar_bar.head(1)

,time,open,close,high,low,volume
2020-01-01 01:22:00,2020-01-01 01:22:00,7188.72,7185.59,7260.43,7174.0,52620.421


In [10]:
from lightweight_charts import Chart

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_dollar_bar[["time", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name="zigzag", style="dotted")
line.set(df_dollar_bar[["time", "zigzag"]])

chart.show(block=True)

2025-03-02 20:42:06.476 python[98390:27395303] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-02 20:42:06.525 python[98390:27395303] +[IMKInputSession subclass]: chose IMKInputSession_Modern


RuntimeError: asyncio.run() cannot be called from a running event loop